In [ ]:
# import torch
# from datasets import load_dataset, Dataset
# from transformers import (
#     T5ForConditionalGeneration,
#     T5Tokenizer,
#     Trainer,
#     TrainingArguments,
#     DataCollatorForSeq2Seq
# )
# import numpy as np
# from sklearn.model_selection import train_test_split

# # 1. Load dataset
# dataset = load_dataset("zjsd/RedStone-QA-mcq")

# # 2. Define preprocessing functions
# def preprocess_for_answer_generation(examples):
#     """Preprocess data for training the model to generate the correct answer letter."""
#     inputs = []
#     for text, question in zip(examples["text"], examples["question"]):
#         combined = f"generate answer: {text} question: {question}"
#         inputs.append(combined)
    
#     # Extract just the letter from "Answer:X" format
#     labels = [answer.replace("Answer:", "") for answer in examples["answer"]]
    
#     return {
#         "input": inputs,
#         "output": labels
#     }

# def preprocess_for_distractor_generation(examples):
#     """Preprocess data for training the model to generate distractors."""
#     inputs = []
#     outputs = []
    
#     for text, question, answer, choices in zip(
#         examples["text"], 
#         examples["question"], 
#         examples["answer"], 
#         examples["choices"]
#     ):
#         # Extract the letter from "Answer:X"
#         answer_letter = answer.replace("Answer:", "")
        
#         # Find the index of the correct answer
#         answer_idx = ord(answer_letter) - ord('A')
        
#         # Get the correct answer text
#         if 0 <= answer_idx < len(choices):
#             correct_answer_text = choices[answer_idx].split('. ', 1)[1] if '. ' in choices[answer_idx] else choices[answer_idx]
            
#             # Create input for distractor generation
#             input_text = f"generate distractors: {text} question: {question} correct answer: {correct_answer_text}"
            
#             # Combine all distractors as output
#             distractors = [choice.split('. ', 1)[1] if '. ' in choice else choice 
#                           for i, choice in enumerate(choices) if i != answer_idx]
#             output_text = " [SEP] ".join(distractors)
            
#             inputs.append(input_text)
#             outputs.append(output_text)
    
#     return {
#         "input": inputs,
#         "output": outputs
#     }

# # 3. Process datasets for both tasks
# answer_dataset = dataset["train"].map(
#     preprocess_for_answer_generation,
#     batched=True,
#     remove_columns=dataset["train"].column_names
# )

# distractor_dataset = dataset["train"].map(
#     preprocess_for_distractor_generation,
#     batched=True,
#     remove_columns=dataset["train"].column_names
# )

# # 4. Split datasets into train and validation
# # Convert to pandas DataFrame for easier splitting
# answer_df = answer_dataset.to_pandas()
# distractor_df = distractor_dataset.to_pandas()

# # Split using pandas DataFrame
# answer_train_df, answer_val_df = train_test_split(answer_df, test_size=0.1, random_state=42)
# distractor_train_df, distractor_val_df = train_test_split(distractor_df, test_size=0.1, random_state=42)

# # Convert back to HuggingFace datasets
# answer_train_dataset = Dataset.from_pandas(answer_train_df)
# answer_val_dataset = Dataset.from_pandas(answer_val_df)
# distractor_train_dataset = Dataset.from_pandas(distractor_train_df)
# distractor_val_dataset = Dataset.from_pandas(distractor_val_df)

# # 5. Load tokenizer and model
# model_name = "google/flan-t5-small"  # You can use other sizes like small, large, xl
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = T5ForConditionalGeneration.from_pretrained(model_name)

# # 6. Define tokenization function
# def tokenize_function(examples):
#     model_inputs = tokenizer(
#         examples["input"],
#         max_length=512,
#         padding="max_length",
#         truncation=True
#     )
    
#     # Tokenize targets
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(
#             examples["output"],
#             max_length=128,
#             padding="max_length",
#             truncation=True
#         )
    
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# # 7. Tokenize datasets
# answer_train_tokenized = answer_train_dataset.map(
#     tokenize_function, 
#     batched=True,
#     remove_columns=["input", "output"]
# )
# answer_val_tokenized = answer_val_dataset.map(
#     tokenize_function, 
#     batched=True,
#     remove_columns=["input", "output"]
# )

# distractor_train_tokenized = distractor_train_dataset.map(
#     tokenize_function, 
#     batched=True,
#     remove_columns=["input", "output"]
# )
# distractor_val_tokenized = distractor_val_dataset.map(
#     tokenize_function, 
#     batched=True,
#     remove_columns=["input", "output"]
# )

# # # 8. Define training arguments for answer generation
# # answer_training_args = TrainingArguments(
# #     output_dir="./results/answer_generation",
# #     evaluation_strategy="epoch",
# #     learning_rate=5e-5,
# #     per_device_train_batch_size=8,
# #     per_device_eval_batch_size=8,
# #     num_train_epochs=3,
# #     weight_decay=0.01,
# #     save_total_limit=3,
# #     load_best_model_at_end=True,
# # )



# answer_training_args = TrainingArguments(
#     output_dir="./results/answer_generation",
#     evaluation_strategy="epoch",  # Make sure this matches save_strategy
#     save_strategy="epoch",  # Change this to "epoch"
#     save_total_limit=2,
#     learning_rate=2e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     load_best_model_at_end=True,  # Keep this since you want the best model at the end
#     metric_for_best_model="eval_loss",  # Specify a metric to track
#     greater_is_better=False,  # Set False since lower loss is better
# )



# # # 9. Define training arguments for distractor generation
# # distractor_training_args = TrainingArguments(
# #     output_dir="./results/distractor_generation",
# #     evaluation_strategy="epoch",
# #     learning_rate=5e-5,
# #     per_device_train_batch_size=8,
# #     per_device_eval_batch_size=8,
# #     num_train_epochs=3,
# #     weight_decay=0.01,
# #     save_total_limit=3,
# #     load_best_model_at_end=True,
# # )
# distractor_training_args = TrainingArguments(
#     output_dir="./results/distractor_generation",
#     evaluation_strategy="epoch",  # Ensure this matches save_strategy
#     save_strategy="epoch",  # Change this to "epoch"
#     learning_rate=5e-5,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     save_total_limit=3,
#     load_best_model_at_end=True,  # Keep this since we want the best model at the end
#     metric_for_best_model="eval_loss",  # Track validation loss
#     greater_is_better=False,  # Lower eval loss is better
# )

# # 10. Define data collator
# data_collator = DataCollatorForSeq2Seq(
#     tokenizer=tokenizer,
#     model=model
# )

# # 11. Train answer generation model
# answer_trainer = Trainer(
#     model=model,
#     args=answer_training_args,
#     train_dataset=answer_train_tokenized,
#     eval_dataset=answer_val_tokenized,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
# )

# print("Training answer generation model...")
# answer_trainer.train()
# answer_trainer.save_model("./results/answer_generation/final_model")

# # 12. Load a fresh model for distractor generation
# distractor_model = T5ForConditionalGeneration.from_pretrained(model_name)

# # 13. Train distractor generation model
# distractor_trainer = Trainer(
#     model=distractor_model,
#     args=distractor_training_args,
#     train_dataset=distractor_train_tokenized,
#     eval_dataset=distractor_val_tokenized,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
# )

# print("Training distractor generation model...")
# distractor_trainer.train()
# distractor_trainer.save_model("./results/distractor_generation/final_model")

# # 14. Inference function for generating MCQs
# def generate_complete_mcq(text, question, answer_model, distractor_model, tokenizer):
#     # Make sure models are in evaluation mode
#     answer_model.eval()
#     distractor_model.eval()
    
#     # Get the device
#     device = next(answer_model.parameters()).device
    
#     # Step 1: Generate the answer letter
#     answer_input = f"generate answer: {text} question: {question}"
#     answer_input_ids = tokenizer(answer_input, return_tensors="pt").input_ids.to(device)
#     answer_outputs = answer_model.generate(answer_input_ids, max_length=10)
#     answer_letter = tokenizer.decode(answer_outputs[0], skip_special_tokens=True)
    
#     # We need the correct answer text for the next step, but in real inference we wouldn't have it
#     # You would need an additional model to generate the correct answer content
#     # For demonstration, let's assume we already have it
#     correct_answer_text = "This is the correct answer content"  # Placeholder
    
#     # Step 2: Generate distractors
#     distractor_input = f"generate distractors: {text} question: {question} correct answer: {correct_answer_text}"
#     distractor_input_ids = tokenizer(distractor_input, return_tensors="pt").input_ids.to(device)
#     distractor_outputs = distractor_model.generate(distractor_input_ids, max_length=256)
#     distractors_text = tokenizer.decode(distractor_outputs[0], skip_special_tokens=True)
    
#     # Split distractors
#     distractors = distractors_text.split(" [SEP] ")
    
#     # Format as MCQ
#     choices = []
#     answer_idx = ord(answer_letter) - ord('A')
    
#     for i in range(len(distractors) + 1):
#         if i == answer_idx:
#             choices.append(f"{chr(65 + i)}. {correct_answer_text}")
#         else:
#             distractor_idx = i if i < answer_idx else i - 1
#             if distractor_idx < len(distractors):
#                 choices.append(f"{chr(65 + i)}. {distractors[distractor_idx]}")
    
#     return {
#         "question": question,
#         "answer": f"Answer:{answer_letter}",
#         "choices": choices
#     }

# print("Training complete! You can now use the trained models to generate MCQs.")

Map:   0%|          | 0/1495442 [00:00<?, ? examples/s]

Map:   0%|          | 0/166161 [00:00<?, ? examples/s]

Map:   0%|          | 0/1495442 [00:00<?, ? examples/s]

Map:   0%|          | 0/166161 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-2-debb5b6f5e79>:212: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  answer_trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Training answer generation model...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

In [ ]:
import torch
from datasets import load_dataset, Dataset
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq
)
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import re
from collections import Counter

# 1. Load dataset
print("Loading dataset...")
dataset = load_dataset("zjsd/RedStone-QA-mcq")
print(f"Dataset loaded with {len(dataset['train'])} examples")

# 2. Define preprocessing functions
def preprocess_for_answer_generation(examples):
    """Preprocess data for training the model to generate the correct answer letter."""
    inputs = []
    for text, question in zip(examples["text"], examples["question"]):
        combined = f"generate answer: {text} question: {question}"
        inputs.append(combined)
    
    # Extract just the letter from "Answer:X" format
    labels = [answer.replace("Answer:", "").strip() for answer in examples["answer"]]
    
    return {
        "input": inputs,
        "output": labels
    }

def preprocess_for_distractor_generation(examples):
    """Preprocess data for training the model to generate distractors."""
    inputs = []
    outputs = []
    
    for text, question, answer, choices in zip(
        examples["text"], 
        examples["question"], 
        examples["answer"], 
        examples["choices"]
    ):
        # Extract the letter from "Answer:X"
        answer_letter = answer.replace("Answer:", "").strip()
        
        # Find the index of the correct answer
        answer_idx = ord(answer_letter) - ord('A')
        
        # Get the correct answer text
        if 0 <= answer_idx < len(choices):
            correct_answer_text = choices[answer_idx].split('. ', 1)[1] if '. ' in choices[answer_idx] else choices[answer_idx]
            
            # Create input for distractor generation
            input_text = f"generate distractors: {text} question: {question} correct answer: {correct_answer_text}"
            
            # Combine all distractors as output
            distractors = [choice.split('. ', 1)[1] if '. ' in choice else choice 
                          for i, choice in enumerate(choices) if i != answer_idx]
            output_text = " [SEP] ".join(distractors)
            
            inputs.append(input_text)
            outputs.append(output_text)
    
    return {
        "input": inputs,
        "output": outputs
    }

# 3. Process datasets for both tasks
print("Preprocessing data for answer generation...")
answer_dataset = dataset["train"].map(
    preprocess_for_answer_generation,
    batched=True,
    remove_columns=dataset["train"].column_names
)

print("Preprocessing data for distractor generation...")
distractor_dataset = dataset["train"].map(
    preprocess_for_distractor_generation,
    batched=True,
    remove_columns=dataset["train"].column_names
)

# 4. Split datasets into train and validation
print("Splitting datasets into train and validation sets...")
# Convert to pandas DataFrame for easier splitting
answer_df = answer_dataset.to_pandas()
distractor_df = distractor_dataset.to_pandas()

# Split using pandas DataFrame
answer_train_df, answer_val_df = train_test_split(answer_df, test_size=0.1, random_state=42)
distractor_train_df, distractor_val_df = train_test_split(distractor_df, test_size=0.1, random_state=42)

print(f"Answer generation: {len(answer_train_df)} training examples, {len(answer_val_df)} validation examples")
print(f"Distractor generation: {len(distractor_train_df)} training examples, {len(distractor_val_df)} validation examples")

# Convert back to HuggingFace datasets
answer_train_dataset = Dataset.from_pandas(answer_train_df)
answer_val_dataset = Dataset.from_pandas(answer_val_df)
distractor_train_dataset = Dataset.from_pandas(distractor_train_df)
distractor_val_dataset = Dataset.from_pandas(distractor_val_df)

# 5. Load tokenizer and model
print("Loading tokenizer and model...")
model_name = "google/flan-t5-small"  # You can use other sizes like small, large, xl
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# 6. Define tokenization function
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["input"],
        max_length=512,
        padding="max_length",
        truncation=True
    )
    
    # Tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["output"],
            max_length=128,
            padding="max_length",
            truncation=True
        )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 7. Tokenize datasets
print("Tokenizing datasets...")
answer_train_tokenized = answer_train_dataset.map(
    tokenize_function, 
    batched=True,
    remove_columns=["input", "output"]
)
answer_val_tokenized = answer_val_dataset.map(
    tokenize_function, 
    batched=True,
    remove_columns=["input", "output"]
)

distractor_train_tokenized = distractor_train_dataset.map(
    tokenize_function, 
    batched=True,
    remove_columns=["input", "output"]
)
distractor_val_tokenized = distractor_val_dataset.map(
    tokenize_function, 
    batched=True,
    remove_columns=["input", "output"]
)

# 8. Define custom metrics computation functions

# Helper function to compute custom ROUGE-like metrics
def compute_ngram_overlap(pred, ref, n=1):
    """Compute n-gram overlap between prediction and reference."""
    def get_ngrams(text, n):
        tokens = text.lower().split()
        ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
        return Counter(ngrams)
    
    pred_ngrams = get_ngrams(pred, n)
    ref_ngrams = get_ngrams(ref, n)
    
    # Find overlap
    overlap = sum((pred_ngrams & ref_ngrams).values())
    
    # Compute precision and recall
    if sum(pred_ngrams.values()) == 0:
        precision = 0
    else:
        precision = overlap / sum(pred_ngrams.values())
        
    if sum(ref_ngrams.values()) == 0:
        recall = 0
    else:
        recall = overlap / sum(ref_ngrams.values())
    
    # Compute F1
    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * precision * recall / (precision + recall)
    
    return precision, recall, f1

def compute_answer_metrics(eval_pred):
    predictions, labels = eval_pred
    # Replace -100 with the pad_token_id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    # Get the predicted tokens
    predicted_tokens = np.argmax(predictions, axis=-1)
    
    # Decode to get the actual letters
    decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in predicted_tokens]
    decoded_labels = [tokenizer.decode(label, skip_special_tokens=True) for label in labels]
    
    # Calculate accuracy
    correct = sum(1 for pred, label in zip(decoded_preds, decoded_labels) if pred.strip() == label.strip())
    accuracy = correct / len(decoded_labels) if len(decoded_labels) > 0 else 0
    
    # Print some examples for debugging
    print("\nAnswer Generation Examples (Prediction, Reference):")
    for i in range(min(5, len(decoded_preds))):
        print(f"  {decoded_preds[i]} | {decoded_labels[i]}")
    
    # Return metrics
    return {
        "accuracy": accuracy,
        "exact_match_ratio": correct / len(decoded_labels) if len(decoded_labels) > 0 else 0,
    }

def compute_distractor_metrics(eval_pred):
    predictions, labels = eval_pred
    # Replace -100 with the pad_token_id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    # Get the predicted tokens
    predicted_tokens = np.argmax(predictions, axis=-1)
    
    # Decode to get the actual distractors
    decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in predicted_tokens]
    decoded_labels = [tokenizer.decode(label, skip_special_tokens=True) for label in labels]
    
    # Calculate our custom ROUGE-like scores
    rouge1_scores = [compute_ngram_overlap(pred, ref, n=1) for pred, ref in zip(decoded_preds, decoded_labels)]
    rouge2_scores = [compute_ngram_overlap(pred, ref, n=2) for pred, ref in zip(decoded_preds, decoded_labels)]
    
    # Average the scores
    rouge1 = np.mean([score[2] for score in rouge1_scores]) if rouge1_scores else 0  # Use F1
    rouge2 = np.mean([score[2] for score in rouge2_scores]) if rouge2_scores else 0  # Use F1
    
    # Print some examples for debugging
    print("\nDistractor Generation Examples (First 100 chars):")
    for i in range(min(3, len(decoded_preds))):
        print(f"  Pred: {decoded_preds[i][:100]}...")
        print(f"  Ref:  {decoded_labels[i][:100]}...")
        print(f"  ROUGE-1 F1: {rouge1_scores[i][2]:.4f}")
        print()
    
    # Return metrics
    return {
        "rouge1": rouge1,
        "rouge2": rouge2,
        "avg_rouge": (rouge1 + rouge2) / 2,
    }

# 9. Define training arguments for answer generation
answer_training_args = TrainingArguments(
    output_dir="./results/answer_generation",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,  # For accuracy, higher is better
    logging_dir="./logs/answer_generation",
    logging_steps=100,  # Log every 100 steps
    report_to=["tensorboard"],  # Report metrics to TensorBoard
)

# 10. Define training arguments for distractor generation
distractor_training_args = TrainingArguments(
    output_dir="./results/distractor_generation",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="avg_rouge",
    greater_is_better=True,  # For ROUGE, higher is better
    logging_dir="./logs/distractor_generation",
    logging_steps=100,  # Log every 100 steps
    report_to=["tensorboard"],  # Report metrics to TensorBoard
)

# 11. Define data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)

# 12. Train answer generation model
print("Initializing answer generation trainer...")
answer_trainer = Trainer(
    model=model,
    args=answer_training_args,
    train_dataset=answer_train_tokenized,
    eval_dataset=answer_val_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_answer_metrics,
)

print("Training answer generation model...")
answer_trainer.train()

# Get validation results
print("Evaluating answer generation model...")
answer_eval_results = answer_trainer.evaluate()
print("\n" + "="*50)
print("ANSWER GENERATION EVALUATION RESULTS")
print("="*50)
print(f"Accuracy: {answer_eval_results['eval_accuracy']:.4f}")
print(f"Exact Match: {answer_eval_results['eval_exact_match_ratio']:.4f}")
print("="*50 + "\n")

answer_trainer.save_model("./results/answer_generation/final_model")

# 13. Load a fresh model for distractor generation
print("Loading fresh model for distractor generation...")
distractor_model = T5ForConditionalGeneration.from_pretrained(model_name)

# 14. Train distractor generation model
print("Initializing distractor generation trainer...")
distractor_trainer = Trainer(
    model=distractor_model,
    args=distractor_training_args,
    train_dataset=distractor_train_tokenized,
    eval_dataset=distractor_val_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_distractor_metrics,
)

print("Training distractor generation model...")
distractor_trainer.train()

# Get validation results
print("Evaluating distractor generation model...")
distractor_eval_results = distractor_trainer.evaluate()
print("\n" + "="*50)
print("DISTRACTOR GENERATION EVALUATION RESULTS")
print("="*50)
print(f"ROUGE-1 F1: {distractor_eval_results['eval_rouge1']:.4f}")
print(f"ROUGE-2 F1: {distractor_eval_results['eval_rouge2']:.4f}")
print(f"Average ROUGE: {distractor_eval_results['eval_avg_rouge']:.4f}")
print("="*50 + "\n")

distractor_trainer.save_model("./results/distractor_generation/final_model")

# 15. Test the models with a simple example
def generate_complete_mcq(context, question, answer_model, distractor_model, tokenizer, num_choices=4):
    # Set models to evaluation mode
    answer_model.eval()
    distractor_model.eval()
    
    # Get the device
    device = next(answer_model.parameters()).device
    
    # Step 1: Generate the answer letter
    answer_input = f"generate answer: {context} question: {question}"
    answer_input_ids = tokenizer(answer_input, return_tensors="pt").input_ids.to(device)
    
    print("Generating answer letter...")
    answer_outputs = answer_model.generate(
        answer_input_ids, 
        max_length=10,
        num_beams=4,
        early_stopping=True
    )
    answer_letter = tokenizer.decode(answer_outputs[0], skip_special_tokens=True).strip()
    print(f"Generated answer letter: {answer_letter}")
    
    # For real inference, you would need another model to generate the correct answer text
    # For this example, we'll use a placeholder
    correct_answer_text = "This is a placeholder for the correct answer content"
    
    # Step 2: Generate distractors
    distractor_input = f"generate distractors: {context} question: {question} correct answer: {correct_answer_text}"
    distractor_input_ids = tokenizer(distractor_input, return_tensors="pt").input_ids.to(device)
    
    print("Generating distractors...")
    distractor_outputs = distractor_model.generate(
        distractor_input_ids, 
        max_length=256,
        num_beams=4,
        early_stopping=True
    )
    distractors_text = tokenizer.decode(distractor_outputs[0], skip_special_tokens=True)
    distractors = distractors_text.split(" [SEP] ")
    print(f"Generated {len(distractors)} distractors")
    
    # Format as MCQ
    choices = []
    answer_idx = ord(answer_letter) - ord('A') if answer_letter.isalpha() else 0
    
    # Ensure we have the right number of choices
    for i in range(num_choices):
        if i == answer_idx:
            choices.append(f"{chr(65 + i)}. {correct_answer_text}")
        else:
            distractor_idx = i if i < answer_idx else i - 1
            if distractor_idx < len(distractors):
                choices.append(f"{chr(65 + i)}. {distractors[distractor_idx]}")
            else:
                choices.append(f"{chr(65 + i)}. Alternative distractor {i+1}")
    
    return {
        "question": question,
        "answer": f"Answer:{answer_letter}",
        "choices": choices
    }

# 16. Final summary and demo
print("\n" + "="*70)
print(" TRAINING SUMMARY ".center(70, "="))
print("="*70)
print(f"Answer Generation Model - Accuracy: {answer_eval_results['eval_accuracy']:.4f}")
print(f"Distractor Generation Model - Avg ROUGE: {distractor_eval_results['eval_avg_rouge']:.4f}")
print("\nModels saved to:")
print(f"  - Answer Generation: ./results/answer_generation/final_model")
print(f"  - Distractor Generation: ./results/distractor_generation/final_model")
print("="*70)

# Demo function to test the models (uncomment to run)
def demo_mcq_generation():
    # Load trained models
    answer_model_path = "./results/answer_generation/final_model"
    distractor_model_path = "./results/distractor_generation/final_model"
    
    print(f"Loading trained models from {answer_model_path} and {distractor_model_path}")
    
    try:
        answer_model = T5ForConditionalGeneration.from_pretrained(answer_model_path)
        distractor_model = T5ForConditionalGeneration.from_pretrained(distractor_model_path)
        
        # Sample context and question
        context = "The Python programming language was created by Guido van Rossum and first released in 1991. It emphasizes code readability with its notable use of significant whitespace."
        question = "Who created the Python programming language?"
        
        # Generate MCQ
        mcq = generate_complete_mcq(context, question, answer_model, distractor_model, tokenizer)
        
        print("\n===== GENERATED MCQ =====")
        print(f"Question: {mcq['question']}")
        print("Choices:")
        for choice in mcq["choices"]:
            print(f"  {choice}")
        print(f"Correct Answer: {mcq['answer']}")
        
    except Exception as e:
        print(f"Error during demo: {e}")
        print("Note: You need to run the training first to generate the model files.")

print("\nTraining complete! The trained models can now be used to generate MCQs.")

# Uncomment to run the demo after training
# demo_mcq_generation()

Loading dataset...


README.md:   0%|          | 0.00/813 [00:00<?, ?B/s]

02f2eecfda5642699cec7306db349dd7.parquet:   0%|          | 0.00/41.3M [00:00<?, ?B/s]

23afc2c9c15646b5b42c3c1fd8191bb3.parquet:   0%|          | 0.00/41.1M [00:00<?, ?B/s]

3e76396d09c3438c82927f7147f79e4e.parquet:   0%|          | 0.00/43.0M [00:00<?, ?B/s]

...

Map:   0%|          | 0/166161 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-2-afe8e6f1b936>:302: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  answer_trainer = Trainer(


Initializing answer generation trainer...
Training answer generation model...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss


In [ ]:
import torch
from datasets import load_dataset, Dataset
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq
)
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import re
from collections import Counter

# 1. Load dataset
print("Loading dataset...")
dataset = load_dataset("zjsd/RedStone-QA-mcq")
print(f"Dataset loaded with {len(dataset['train'])} examples")

# Reduce dataset size by taking only 20% of the examples
print("Reducing dataset size to 20% of original...")
# Create a smaller sample using the take method
reduced_size = int(len(dataset['train']) * 0.2)
dataset['train'] = dataset['train'].shuffle(seed=42).select(range(reduced_size))
print(f"Reduced dataset size: {len(dataset['train'])} examples")

# 2. Define preprocessing functions
def preprocess_for_answer_generation(examples):
    """Preprocess data for training the model to generate the correct answer letter."""
    inputs = []
    for text, question in zip(examples["text"], examples["question"]):
        combined = f"generate answer: {text} question: {question}"
        inputs.append(combined)
    
    # Extract just the letter from "Answer:X" format
    labels = [answer.replace("Answer:", "").strip() for answer in examples["answer"]]
    
    return {
        "input": inputs,
        "output": labels
    }

def preprocess_for_distractor_generation(examples):
    """Preprocess data for training the model to generate distractors."""
    inputs = []
    outputs = []
    
    for text, question, answer, choices in zip(
        examples["text"], 
        examples["question"], 
        examples["answer"], 
        examples["choices"]
    ):
        # Extract the letter from "Answer:X"
        answer_letter = answer.replace("Answer:", "").strip()
        
        # Find the index of the correct answer
        answer_idx = ord(answer_letter) - ord('A')
        
        # Get the correct answer text
        if 0 <= answer_idx < len(choices):
            correct_answer_text = choices[answer_idx].split('. ', 1)[1] if '. ' in choices[answer_idx] else choices[answer_idx]
            
            # Create input for distractor generation
            input_text = f"generate distractors: {text} question: {question} correct answer: {correct_answer_text}"
            
            # Combine all distractors as output
            distractors = [choice.split('. ', 1)[1] if '. ' in choice else choice 
                          for i, choice in enumerate(choices) if i != answer_idx]
            output_text = " [SEP] ".join(distractors)
            
            inputs.append(input_text)
            outputs.append(output_text)
    
    return {
        "input": inputs,
        "output": outputs
    }

# 3. Process datasets for both tasks
print("Preprocessing data for answer generation...")
answer_dataset = dataset["train"].map(
    preprocess_for_answer_generation,
    batched=True,
    remove_columns=dataset["train"].column_names
)

print("Preprocessing data for distractor generation...")
distractor_dataset = dataset["train"].map(
    preprocess_for_distractor_generation,
    batched=True,
    remove_columns=dataset["train"].column_names
)

# 4. Split datasets into train and validation
print("Splitting datasets into train and validation sets...")
# Convert to pandas DataFrame for easier splitting
answer_df = answer_dataset.to_pandas()
distractor_df = distractor_dataset.to_pandas()

# Split using pandas DataFrame
answer_train_df, answer_val_df = train_test_split(answer_df, test_size=0.1, random_state=42)
distractor_train_df, distractor_val_df = train_test_split(distractor_df, test_size=0.1, random_state=42)

print(f"Answer generation: {len(answer_train_df)} training examples, {len(answer_val_df)} validation examples")
print(f"Distractor generation: {len(distractor_train_df)} training examples, {len(distractor_val_df)} validation examples")

# Convert back to HuggingFace datasets
answer_train_dataset = Dataset.from_pandas(answer_train_df)
answer_val_dataset = Dataset.from_pandas(answer_val_df)
distractor_train_dataset = Dataset.from_pandas(distractor_train_df)
distractor_val_dataset = Dataset.from_pandas(distractor_val_df)

# 5. Load tokenizer and model
print("Loading tokenizer and model...")
model_name = "google/flan-t5-small"  # Using the small model for faster training
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# 6. Define tokenization function
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["input"],
        max_length=512,
        padding="max_length",
        truncation=True
    )
    
    # Tokenize targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["output"],
            max_length=128,
            padding="max_length",
            truncation=True
        )
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 7. Tokenize datasets
print("Tokenizing datasets...")
answer_train_tokenized = answer_train_dataset.map(
    tokenize_function, 
    batched=True,
    remove_columns=["input", "output"]
)
answer_val_tokenized = answer_val_dataset.map(
    tokenize_function, 
    batched=True,
    remove_columns=["input", "output"]
)

distractor_train_tokenized = distractor_train_dataset.map(
    tokenize_function, 
    batched=True,
    remove_columns=["input", "output"]
)
distractor_val_tokenized = distractor_val_dataset.map(
    tokenize_function, 
    batched=True,
    remove_columns=["input", "output"]
)

# 8. Define custom metrics computation functions

# Helper function to compute custom ROUGE-like metrics
def compute_ngram_overlap(pred, ref, n=1):
    """Compute n-gram overlap between prediction and reference."""
    def get_ngrams(text, n):
        tokens = text.lower().split()
        ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
        return Counter(ngrams)
    
    pred_ngrams = get_ngrams(pred, n)
    ref_ngrams = get_ngrams(ref, n)
    
    # Find overlap
    overlap = sum((pred_ngrams & ref_ngrams).values())
    
    # Compute precision and recall
    if sum(pred_ngrams.values()) == 0:
        precision = 0
    else:
        precision = overlap / sum(pred_ngrams.values())
        
    if sum(ref_ngrams.values()) == 0:
        recall = 0
    else:
        recall = overlap / sum(ref_ngrams.values())
    
    # Compute F1
    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * precision * recall / (precision + recall)
    
    return precision, recall, f1

def compute_answer_metrics(eval_pred):
    predictions, labels = eval_pred
    # Replace -100 with the pad_token_id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    # Get the predicted tokens
    predicted_tokens = np.argmax(predictions, axis=-1)
    
    # Decode to get the actual letters
    decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in predicted_tokens]
    decoded_labels = [tokenizer.decode(label, skip_special_tokens=True) for label in labels]
    
    # Calculate accuracy
    correct = sum(1 for pred, label in zip(decoded_preds, decoded_labels) if pred.strip() == label.strip())
    accuracy = correct / len(decoded_labels) if len(decoded_labels) > 0 else 0
    
    # Print some examples for debugging
    print("\nAnswer Generation Examples (Prediction, Reference):")
    for i in range(min(5, len(decoded_preds))):
        print(f"  {decoded_preds[i]} | {decoded_labels[i]}")
    
    # Return metrics
    return {
        "accuracy": accuracy,
        "exact_match_ratio": correct / len(decoded_labels) if len(decoded_labels) > 0 else 0,
    }

def compute_distractor_metrics(eval_pred):
    predictions, labels = eval_pred
    # Replace -100 with the pad_token_id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    
    # Get the predicted tokens
    predicted_tokens = np.argmax(predictions, axis=-1)
    
    # Decode to get the actual distractors
    decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in predicted_tokens]
    decoded_labels = [tokenizer.decode(label, skip_special_tokens=True) for label in labels]
    
    # Calculate our custom ROUGE-like scores
    rouge1_scores = [compute_ngram_overlap(pred, ref, n=1) for pred, ref in zip(decoded_preds, decoded_labels)]
    rouge2_scores = [compute_ngram_overlap(pred, ref, n=2) for pred, ref in zip(decoded_preds, decoded_labels)]
    
    # Average the scores
    rouge1 = np.mean([score[2] for score in rouge1_scores]) if rouge1_scores else 0  # Use F1
    rouge2 = np.mean([score[2] for score in rouge2_scores]) if rouge2_scores else 0  # Use F1
    
    # Print some examples for debugging
    print("\nDistractor Generation Examples (First 100 chars):")
    for i in range(min(3, len(decoded_preds))):
        print(f"  Pred: {decoded_preds[i][:100]}...")
        print(f"  Ref:  {decoded_labels[i][:100]}...")
        print(f"  ROUGE-1 F1: {rouge1_scores[i][2]:.4f}")
        print()
    
    # Return metrics
    return {
        "rouge1": rouge1,
        "rouge2": rouge2,
        "avg_rouge": (rouge1 + rouge2) / 2,
    }

# 9. Define training arguments for answer generation with reduced epochs
answer_training_args = TrainingArguments(
    output_dir="./results/answer_generation",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Increased batch size
    per_device_eval_batch_size=8,   # Increased batch size
    num_train_epochs=2,             # Reduced from 3 to 2
    weight_decay=0.01,
    save_total_limit=1,             # Reduced from 2 to 1
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    logging_dir="./logs/answer_generation",
    logging_steps=100,
    report_to=["tensorboard"],
)

# 10. Define training arguments for distractor generation with reduced epochs
distractor_training_args = TrainingArguments(
    output_dir="./results/distractor_generation",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,  # Increased batch size
    per_device_eval_batch_size=16,   # Increased batch size
    num_train_epochs=3,              # Reduced from 3 to 2
    weight_decay=0.01,
    save_total_limit=1,              # Reduced from 3 to 1
    load_best_model_at_end=True,
    metric_for_best_model="avg_rouge",
    greater_is_better=True,
    logging_dir="./logs/distractor_generation",
    logging_steps=100,
    report_to=["tensorboard"],
)

# 11. Define data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)

# 12. Train answer generation model
print("Initializing answer generation trainer...")
answer_trainer = Trainer(
    model=model,
    args=answer_training_args,
    train_dataset=answer_train_tokenized,
    eval_dataset=answer_val_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_answer_metrics,
)

print("Training answer generation model...")
answer_trainer.train()

# Get validation results
print("Evaluating answer generation model...")
answer_eval_results = answer_trainer.evaluate()
print("\n" + "="*50)
print("ANSWER GENERATION EVALUATION RESULTS")
print("="*50)
print(f"Accuracy: {answer_eval_results['eval_accuracy']:.4f}")
print(f"Exact Match: {answer_eval_results['eval_exact_match_ratio']:.4f}")
print("="*50 + "\n")

answer_trainer.save_model("./results/answer_generation/final_model")

# 13. Load a fresh model for distractor generation
print("Loading fresh model for distractor generation...")
distractor_model = T5ForConditionalGeneration.from_pretrained(model_name)

# 14. Train distractor generation model
print("Initializing distractor generation trainer...")
distractor_trainer = Trainer(
    model=distractor_model,
    args=distractor_training_args,
    train_dataset=distractor_train_tokenized,
    eval_dataset=distractor_val_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_distractor_metrics,
)

print("Training distractor generation model...")
distractor_trainer.train()

# Get validation results
print("Evaluating distractor generation model...")
distractor_eval_results = distractor_trainer.evaluate()
print("\n" + "="*50)
print("DISTRACTOR GENERATION EVALUATION RESULTS")
print("="*50)
print(f"ROUGE-1 F1: {distractor_eval_results['eval_rouge1']:.4f}")
print(f"ROUGE-2 F1: {distractor_eval_results['eval_rouge2']:.4f}")
print(f"Average ROUGE: {distractor_eval_results['eval_avg_rouge']:.4f}")
print("="*50 + "\n")

distractor_trainer.save_model("./results/distractor_generation/final_model")

# 15. Test the models with a simple example
def generate_complete_mcq(context, question, answer_model, distractor_model, tokenizer, num_choices=4):
    # Set models to evaluation mode
    answer_model.eval()
    distractor_model.eval()
    
    # Get the device
    device = next(answer_model.parameters()).device
    
    # Step 1: Generate the answer letter
    answer_input = f"generate answer: {context} question: {question}"
    answer_input_ids = tokenizer(answer_input, return_tensors="pt").input_ids.to(device)
    
    print("Generating answer letter...")
    answer_outputs = answer_model.generate(
        answer_input_ids, 
        max_length=10,
        num_beams=4,
        early_stopping=True
    )
    answer_letter = tokenizer.decode(answer_outputs[0], skip_special_tokens=True).strip()
    print(f"Generated answer letter: {answer_letter}")
    
    # For real inference, you would need another model to generate the correct answer text
    # For this example, we'll use a placeholder
    correct_answer_text = "This is a placeholder for the correct answer content"
    
    # Step 2: Generate distractors
    distractor_input = f"generate distractors: {context} question: {question} correct answer: {correct_answer_text}"
    distractor_input_ids = tokenizer(distractor_input, return_tensors="pt").input_ids.to(device)
    
    print("Generating distractors...")
    distractor_outputs = distractor_model.generate(
        distractor_input_ids, 
        max_length=256,
        num_beams=4,
        early_stopping=True
    )
    distractors_text = tokenizer.decode(distractor_outputs[0], skip_special_tokens=True)
    distractors = distractors_text.split(" [SEP] ")
    print(f"Generated {len(distractors)} distractors")
    
    # Format as MCQ
    choices = []
    answer_idx = ord(answer_letter) - ord('A') if answer_letter.isalpha() else 0
    
    # Ensure we have the right number of choices
    for i in range(num_choices):
        if i == answer_idx:
            choices.append(f"{chr(65 + i)}. {correct_answer_text}")
        else:
            distractor_idx = i if i < answer_idx else i - 1
            if distractor_idx < len(distractors):
                choices.append(f"{chr(65 + i)}. {distractors[distractor_idx]}")
            else:
                choices.append(f"{chr(65 + i)}. Alternative distractor {i+1}")
    
    return {
        "question": question,
        "answer": f"Answer:{answer_letter}",
        "choices": choices
    }

# 16. Final summary and demo
print("\n" + "="*70)
print(" TRAINING SUMMARY ".center(70, "="))
print("="*70)
print(f"Answer Generation Model - Accuracy: {answer_eval_results['eval_accuracy']:.4f}")
print(f"Distractor Generation Model - Avg ROUGE: {distractor_eval_results['eval_avg_rouge']:.4f}")
print("\nModels saved to:")
print(f"  - Answer Generation: ./results/answer_generation/final_model")
print(f"  - Distractor Generation: ./results/distractor_generation/final_model")
print("="*70)

# Demo function to test the models (uncomment to run)
def demo_mcq_generation():
    # Load trained models
    answer_model_path = "./results/answer_generation/final_model"
    distractor_model_path = "./results/distractor_generation/final_model"
    
    print(f"Loading trained models from {answer_model_path} and {distractor_model_path}")
    
    try:
        answer_model = T5ForConditionalGeneration.from_pretrained(answer_model_path)
        distractor_model = T5ForConditionalGeneration.from_pretrained(distractor_model_path)
        
        # Sample context and question
        context = "The Python programming language was created by Guido van Rossum and first released in 1991. It emphasizes code readability with its notable use of significant whitespace."
        question = "Who created the Python programming language?"
        
        # Generate MCQ
        mcq = generate_complete_mcq(context, question, answer_model, distractor_model, tokenizer)
        
        print("\n===== GENERATED MCQ =====")
        print(f"Question: {mcq['question']}")
        print("Choices:")
        for choice in mcq["choices"]:
            print(f"  {choice}")
        print(f"Correct Answer: {mcq['answer']}")
        
    except Exception as e:
        print(f"Error during demo: {e}")
        print("Note: You need to run the training first to generate the model files.")

print("\nTraining complete! The trained models can now be used to generate MCQs.")

# Uncomment to run the demo after training
# demo_mcq_generation()

Loading dataset...


README.md:   0%|          | 0.00/813 [00:00<?, ?B/s]

02f2eecfda5642699cec7306db349dd7.parquet:   0%|          | 0.00/41.3M [00:00<?, ?B/s]

23afc2c9c15646b5b42c3c1fd8191bb3.parquet:   0%|          | 0.00/41.1M [00:00<?, ?B/s]

3e76396d09c3438c82927f7147f79e4e.parquet:   0%|          | 0.00/43.0M [00:00<?, ?B/s]

...

Map:   0%|          | 0/33232 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-1-803e05ab98e4>:309: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  answer_trainer = Trainer(


Initializing answer generation trainer...
Training answer generation model...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
